In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean-business-csv-1/clean_business_csv_1
/kaggle/input/yelpdb/__notebook_source__.ipynb
/kaggle/input/yelpdb/database.db


In [2]:
#importing necessary libraries
import numpy as np 
import pandas as pd
import matplotlib as plt
import tensorflow as tf
import json 
import os
import csv
import json
import sqlite3
from pathlib import Path
from typing import Dict, Text
from collections import defaultdict
%matplotlib inline

In [3]:
#Function to convert query(which comes in as a list of tuples to DataFrame)
def toDf(res,cols):
    try:
        return pd.DataFrame(res,columns=cols)
    except Exception as e:
        print(f'The DataFrame cannot be created because of this error: {e}')

In [4]:
#Function to connect to database
def connect(path):
    con=sqlite3.connect(path)
    return con

#function to create a query
def createQuery(query,connection):
    try:
        connection.cursor().execute(query)
        print('Table Created')
    except Exception as e:
        print('Exception',e)

#Function to perform get queries
def getQuery(query,connection,cols):
    try:
        result=connection.cursor().execute(query)
        print('Query Executed')
        return toDf(result.fetchall(),cols)
    except Exception as e:
        print('Exception',e)

In [5]:
# connecting to the database
con = connect('../input/yelpdb/database.db')

# creating a list of the business cloumns
business_cols = ['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours']

# fetching columns from the business table
business = getQuery('Select * from business',con,business_cols)

# dropping a few columns
business.drop(['name','address','postal_code','city','state','hours'], axis=1, inplace=True)

# dropping duplicate rows if any
business = business.drop_duplicates()

#we are considering only businesses which just serves food or is a restraunt
temp = business.categories.str.contains('Restaurant', na=False) | business.categories.str.contains('Food', na=False)
business = business[temp].reset_index()

# having a quick look at the business tables
business.head()

Query Executed


,index,business_id,latitude,longitude,stars,review_count,is_open,attributes,categories
0,0,6iYb2HFDywm3zjuRg0shjw,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': ""u...","Gastropubs, Food, Beer Gardens, Restaurants, B..."
1,1,tCbdrRPZA0oiIYSmHG3J0w,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C..."
2,2,D4JtQNTI4X3KcbzacDJsMw,49.251342,-123.101333,3.5,169,1,"{'GoodForKids': 'True', 'Alcohol': ""u'none'"", ...","Restaurants, Thai"
3,3,jFYIsSb7r1QeESVUnXPHBw,39.947007,-82.997471,4.5,11,1,None,"Breakfast & Brunch, Restaurants"
4,4,rYs_1pNB_RMtn5WQh55QDA,39.998449,-105.281006,3.5,5,1,"{'BikeParking': 'True', 'RestaurantsTakeOut': ...","Food, Shopping, Convenience Stores, Souvenir S..."


In [6]:
#creating a function called create categories 
def createCategories(row):
    if business['categories'][row]:
        return(business['categories'][row].split(','))
    return ""

In [7]:
# creating a set of unique categories
uniquecat =set()
for i in range(len(business)):
    uniquecat.update(list(createCategories(i)))
# creating unique categories by using strip to avoid duplicates
uniquecategories = set()
for i in uniquecat:
    uniquecategories.add(i.strip())

In [8]:
# creating a dictionary to count the number of times a certain category appears in the whole categories column
categoriecount = {}
for i in business['categories']:
    m = i.split(",")
    for j in m:
        j = j.strip()
        if j not in categoriecount.keys():
            categoriecount[j] = 1
        else:
            categoriecount[j] = categoriecount[j] + 1

In [9]:
# creating a dictionary of all the categories which we want to add. We have only kept those categories which appear in atleast 1% of the records
categoriestoadd = {}
for key,value in categoriecount.items():
    per = (value/63915)*100
    if per>=1:
        categoriestoadd[key] = value

**We then created a dataframe columns_clean_business which has business_id, latitude, longitude, stars, review_count, is_open and all the columns from the categoriestoadd dictionary. We have commented this because it takes a lot of time to execute and we have saved the results in clean-business-csv-1**

In [10]:
# columns_clean_business = ['business_id','latitude','longitude','stars','review_count','is_open']
# columns_clean_business = columns_clean_business + list(categoriestoadd.keys())

# clean_business = pd.DataFrame(columns=columns_clean_business)

# clean_business['business_id'] = business['business_id'] 
# clean_business['latitude'] = business['latitude']
# clean_business['longitude'] = business['longitude']
# clean_business['stars'] = business['stars']
# clean_business['review_count'] = business['review_count']
# clean_business['is_open'] = business['is_open']


# for i in range(len(business['categories'])):
#     for j in list(categoriestoadd.keys()):
#         m = business['categories'][i].split(",")
#         if j in m:
#             clean_business[j][i] = 1
#         else:
#             clean_business[j][i] = 0

# clean_business.to_csv('clean_business_csv_1')

In [11]:
#importing the clean-business-csv-1 file
clean_business = pd.read_csv('../input/clean-business-csv-1/clean_business_csv_1')

#having a quick look at clean-business
clean_business.head()

,Unnamed: 0,business_id,latitude,longitude,stars,review_count,is_open,Food,Restaurants,Bars,...,Middle Eastern,Wine Bars,Latin American,Lounges,Drugstores,Hot Dogs,Bubble Tea,Noodles,Comfort Food,Street Vendors
0,0,6iYb2HFDywm3zjuRg0shjw,40.017544,-105.283348,4.0,86,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,tCbdrRPZA0oiIYSmHG3J0w,45.588906,-122.593331,4.0,126,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,D4JtQNTI4X3KcbzacDJsMw,49.251342,-123.101333,3.5,169,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,jFYIsSb7r1QeESVUnXPHBw,39.947007,-82.997471,4.5,11,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,rYs_1pNB_RMtn5WQh55QDA,39.998449,-105.281006,3.5,5,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


**Now we will concatenate the unique attributes.**

In [12]:
# creating a function called getAttribute which fetches the attributes in that particular row
def getAttributes(row):
        if business['attributes'][row]:
            string = business['attributes'][row]
            w = eval(string)
            return(w.keys())
        return ""
    
# finding the unique attributes
uniqueattributes =set()
for i in range(len(business)):
    uniqueattributes.update(list(getAttributes(i)))
    
def createcolumnvalues(colname):
    colvalue = []
    for i in range(len(business['attributes'])):
        string = business['attributes'][i]
        if string:
            w = eval(string)
            if colname in w.keys():
                colvalue.append(w[colname])
            else:
                colvalue.append(np.nan)
        else:
            colvalue.append(np.nan)
    return colvalue

for colname in uniqueattributes:
    l = createcolumnvalues(colname)
    clean_business[colname] = l

# dropping the unnamed column
clean_business.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
# number of unique attributes
print("the number of unique attributes is:"+str(len(uniqueattributes)))
# name of the unique attributes
print(clean_business.columns[-39:])

the number of unique attributes is:39
Index(['DietaryRestrictions', 'DogsAllowed', 'RestaurantsAttire', 'Smoking',
       'BYOBCorkage', 'Ambience', 'RestaurantsCounterService',
       'RestaurantsPriceRange2', 'AgesAllowed', 'DriveThru', 'Caters',
       'GoodForMeal', 'AcceptsInsurance', 'BestNights', 'Corkage',
       'RestaurantsGoodForGroups', 'Open24Hours', 'WiFi',
       'RestaurantsDelivery', 'GoodForDancing', 'Music',
       'BusinessAcceptsCreditCards', 'BusinessParking',
       'BusinessAcceptsBitcoin', 'RestaurantsTakeOut', 'NoiseLevel',
       'HairSpecializesIn', 'RestaurantsTableService', 'BikeParking',
       'RestaurantsReservations', 'Alcohol', 'HasTV', 'GoodForKids',
       'CoatCheck', 'ByAppointmentOnly', 'OutdoorSeating', 'BYOB', 'HappyHour',
       'WheelchairAccessible'],
      dtype='object')


#### We explored all the columns above and dropped them if they had nan values for more than 80% of the records. 

In [14]:
# all the columns mentioned below are the ones which have nan values for more than 80%
clean_business.drop(['Corkage','Smoking','GoodForDancing','ByAppointmentOnly','Open24Hours','BYOB','AgesAllowed','HairSpecializesIn','RestaurantsCounterService','DietaryRestrictions','BYOBCorkage','BusinessAcceptsBitcoin','Music','AcceptsInsurance','BestNights'],axis=1,inplace=True)

In [15]:
# all the columns mentioned below have the unique values: False, None, NaN and True. We have replace them with true or false
clean_business['WheelchairAccessible'] = clean_business['WheelchairAccessible'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['RestaurantsTableService'] = clean_business['RestaurantsTableService'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['HasTV'] = clean_business['HasTV'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['BusinessAcceptsCreditCards'] = clean_business['BusinessAcceptsCreditCards'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['CoatCheck'] = clean_business['CoatCheck'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['RestaurantsTakeOut'] = clean_business['RestaurantsTakeOut'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['GoodForKids'] = clean_business['GoodForKids'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['OutdoorSeating'] = clean_business['OutdoorSeating'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['RestaurantsGoodForGroups'] = clean_business['RestaurantsGoodForGroups'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['DriveThru'] = clean_business['DriveThru'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['HappyHour'] = clean_business['HappyHour'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['GoodForKids'] = clean_business['GoodForKids'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['BikeParking'] = clean_business['BikeParking'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['RestaurantsTableService'] = clean_business['RestaurantsTableService'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['RestaurantsDelivery'] = clean_business['RestaurantsDelivery'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['DogsAllowed'] = clean_business['DogsAllowed'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['RestaurantsReservations'] = clean_business['RestaurantsReservations'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['GoodForKids'] = clean_business['GoodForKids'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['Caters'] = clean_business['Caters'].replace(['False','None',np.nan],False).replace('True',True)

#### Replacing the alcohol column into 2 columns 'full_bar' and 'beer_and_wine' </br>Alcohol column is in the form of strings link: u'full_bar'.So we use the eval function to correct this format.

In [16]:
# replacing the alcohol column into 2 columns 'full_bar' and 'beer_and_wine' 
# we will then fill the values with 0s and 1s.
uniqueattributes =set()
for i in range(len(clean_business)):
    if (str(clean_business['Alcohol'][i]) != 'nan') and (str(clean_business['Alcohol'][i]) != 'None'):
        string = clean_business['Alcohol'][i]
        w = eval(string)
        uniqueattributes.add(w)
        
colvalue = []
for i in range(len(clean_business['Alcohol'])):
    string = clean_business['Alcohol'][i]
    if (str(clean_business['Alcohol'][i]) != 'nan') and (str(clean_business['Alcohol'][i]) != 'None'):
        w = eval(string)
        colvalue.append(w)
    else:
        colvalue.append(np.nan)
        
alc = pd.get_dummies(np.array(colvalue)).drop(['nan','none'],axis=1)

#### Replacing the Ambience column into 'classy','upscale','trendy','divey','touristy','romantic','casual','intimate' and 'hipster'</br> Ambience column is in the form of dictionary like: </br>"{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}"</br>.So we use the createcolumnvalues_1 function to correct this format.

In [17]:
# creating a function which creates a list of values which will be used when appending that column in the table. 
# We are handling values in the format like
# "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}"
def createcolumnvalues_1(colname,parentcolumnname):
    colvalue = []
    for i in range(len(clean_business[parentcolumnname])):
        string = clean_business[parentcolumnname][i]
        if (str(clean_business[parentcolumnname][i]) != 'nan') and (str(clean_business[parentcolumnname][i]) != 'None'):
            w = eval(string)
            if colname in w.keys():
                colvalue.append(w[colname])
            else:
                colvalue.append(np.nan)
        else:
            colvalue.append(np.nan)
    return colvalue

In [18]:
# creating a list with all types of ambiences
uniqueattributes = ['classy','upscale','trendy','divey','touristy','romantic','casual','intimate','hipster']

for colname in uniqueattributes:
    l = createcolumnvalues_1(colname,'Ambience')
    clean_business[colname] = l
    
pd.concat((clean_business,alc),axis=1)

clean_business['classy'] = clean_business['classy'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['upscale'] = clean_business['upscale'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['trendy'] = clean_business['trendy'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['divey'] = clean_business['divey'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['touristy'] = clean_business['touristy'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['romantic'] = clean_business['romantic'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['casual'] = clean_business['casual'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['intimate'] = clean_business['intimate'].replace(['False','None',np.nan],False).replace('True',True)
clean_business['hipster'] = clean_business['hipster'].replace(['False','None',np.nan],False).replace('True',True)

#### Replacing the BusinessParking column into 'garage','street','lot' and 'valet'</br>Ambience column is in the form of dictionary like:</br>"{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"</br>So we use the createcolumnvalues_1 function to correct this format

In [19]:
for colname in uniqueattributes:
    l = createcolumnvalues_1(colname,'BusinessParking')
    clean_business[colname] = l
    
for i in uniqueattributes:
    clean_business[i] = clean_business[i].replace(['False','None',np.nan],False).replace('True',True)

#### Replacing the WiFi column into 'free', 'no' and 'paid'</br>WiFi column is in the form of dictionary like:</br>"u'free'"</br>So we use the eval function to correct this format

In [20]:
uniqueattributes =set()
for i in range(len(clean_business)):
    if (str(clean_business['WiFi'][i]) != 'nan') and (str(clean_business['WiFi'][i]) != 'None'):
        string = clean_business['WiFi'][i]
        w = eval(string)
        uniqueattributes.add(w)
        
colvalue = []
for i in range(len(clean_business['WiFi'])):
    string = clean_business['WiFi'][i]
    if (str(clean_business['WiFi'][i]) != 'nan') and (str(clean_business['WiFi'][i]) != 'None'):
        w = eval(string)
        colvalue.append(w)
    else:
        colvalue.append(np.nan)
        
wi = pd.get_dummies(np.array(colvalue)).drop('nan',axis=1)

pd.concat((clean_business,wi),axis=1)

,business_id,latitude,longitude,stars,review_count,is_open,Food,Restaurants,Bars,American (Traditional),...,trendy,divey,touristy,romantic,casual,intimate,hipster,free,no,paid
0,6iYb2HFDywm3zjuRg0shjw,40.017544,-105.283348,4.0,86,1,0,0,0,0,...,False,False,False,False,False,False,False,1,0,0
1,tCbdrRPZA0oiIYSmHG3J0w,45.588906,-122.593331,4.0,126,1,0,0,0,0,...,False,False,False,False,False,False,False,1,0,0
2,D4JtQNTI4X3KcbzacDJsMw,49.251342,-123.101333,3.5,169,1,0,1,0,0,...,False,False,False,False,False,False,False,0,1,0
3,jFYIsSb7r1QeESVUnXPHBw,39.947007,-82.997471,4.5,11,1,0,0,0,0,...,False,False,False,False,False,False,False,0,0,0
4,rYs_1pNB_RMtn5WQh55QDA,39.998449,-105.281006,3.5,5,1,1,0,0,0,...,False,False,False,False,False,False,False,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63910,Q78fYV6B6P6GmX07YVgi4g,30.304761,-97.937394,3.0,310,1,0,0,0,0,...,False,False,False,False,False,False,False,1,0,0
63911,uXdQkuEtvLAzfc3MsO-sTQ,33.810996,-84.367084,3.0,185,1,0,1,0,0,...,False,False,False,False,False,False,False,0,1,0
63912,bQX-kwVTyZgcdZGEPzce6Q,28.499836,-81.047478,4.5,18,1,0,0,0,0,...,False,False,False,False,False,False,False,0,0,0
63913,GB75wPibj3IjNauaoCxyGA,45.529647,-122.685153,4.0,10,1,0,0,0,0,...,False,False,False,False,False,False,False,0,0,0


#### Replacing the RestaurantsPriceRange2 column into 1, 2, 3 and 4.</br>RestaurantsPriceRange2 column is in the form of dictionary like:</br>'2'</br>So we use the eval function to correct this format

In [21]:
def isNaN(num):
    return num!= num

uniqueattributes =set()
for i in range(len(clean_business)):
    flag = isNaN(clean_business['RestaurantsPriceRange2'][i])
    if not flag:
        string = clean_business['RestaurantsPriceRange2'][i]
        w = eval(string)
        uniqueattributes.add(w)
        
colvalue = []
for i in range(len(clean_business['RestaurantsPriceRange2'])):
    string = clean_business['RestaurantsPriceRange2'][i]
    if (str(clean_business['RestaurantsPriceRange2'][i]) != 'nan') and (str(clean_business['RestaurantsPriceRange2'][i]) != 'None'):
        w = eval(string)
        colvalue.append(w)
    else:
        colvalue.append(np.nan)
        
rpr2 = pd.get_dummies(np.array(colvalue))

pd.concat((clean_business,rpr2),axis=1)

,business_id,latitude,longitude,stars,review_count,is_open,Food,Restaurants,Bars,American (Traditional),...,divey,touristy,romantic,casual,intimate,hipster,1.0,2.0,3.0,4.0
0,6iYb2HFDywm3zjuRg0shjw,40.017544,-105.283348,4.0,86,1,0,0,0,0,...,False,False,False,False,False,False,0,1,0,0
1,tCbdrRPZA0oiIYSmHG3J0w,45.588906,-122.593331,4.0,126,1,0,0,0,0,...,False,False,False,False,False,False,0,1,0,0
2,D4JtQNTI4X3KcbzacDJsMw,49.251342,-123.101333,3.5,169,1,0,1,0,0,...,False,False,False,False,False,False,0,1,0,0
3,jFYIsSb7r1QeESVUnXPHBw,39.947007,-82.997471,4.5,11,1,0,0,0,0,...,False,False,False,False,False,False,0,0,0,0
4,rYs_1pNB_RMtn5WQh55QDA,39.998449,-105.281006,3.5,5,1,1,0,0,0,...,False,False,False,False,False,False,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63910,Q78fYV6B6P6GmX07YVgi4g,30.304761,-97.937394,3.0,310,1,0,0,0,0,...,False,False,False,False,False,False,0,1,0,0
63911,uXdQkuEtvLAzfc3MsO-sTQ,33.810996,-84.367084,3.0,185,1,0,1,0,0,...,False,False,False,False,False,False,0,1,0,0
63912,bQX-kwVTyZgcdZGEPzce6Q,28.499836,-81.047478,4.5,18,1,0,0,0,0,...,False,False,False,False,False,False,0,1,0,0
63913,GB75wPibj3IjNauaoCxyGA,45.529647,-122.685153,4.0,10,1,0,0,0,0,...,False,False,False,False,False,False,0,0,0,0


#### Replacing the RestaurantsAttire column into 1, 2, 3 and 4.</br>RestaurantsAttire column is in the format like:</br>"'casual'"</br>So we use the eval function to correct this format

In [22]:
uniqueattributes =set()
for i in range(len(clean_business)):
    if (str(clean_business['RestaurantsAttire'][i]) != 'nan') and (str(clean_business['RestaurantsAttire'][i]) != 'None'):
        string = clean_business['RestaurantsAttire'][i]
        w = eval(string)
        uniqueattributes.add(w)
        
colvalue = []
for i in range(len(clean_business['RestaurantsAttire'])):
    string = clean_business['RestaurantsAttire'][i]
    if (str(clean_business['RestaurantsAttire'][i]) != 'nan') and (str(clean_business['RestaurantsAttire'][i]) != 'None'):
        w = eval(string)
        colvalue.append(w)
    else:
        colvalue.append(np.nan)
        
att = pd.get_dummies(np.array(colvalue)).drop('nan',axis=1)

pd.concat((clean_business,att),axis=1)

,business_id,latitude,longitude,stars,review_count,is_open,Food,Restaurants,Bars,American (Traditional),...,trendy,divey,touristy,romantic,casual,intimate,hipster,casual,dressy,formal
0,6iYb2HFDywm3zjuRg0shjw,40.017544,-105.283348,4.0,86,1,0,0,0,0,...,False,False,False,False,False,False,False,1,0,0
1,tCbdrRPZA0oiIYSmHG3J0w,45.588906,-122.593331,4.0,126,1,0,0,0,0,...,False,False,False,False,False,False,False,1,0,0
2,D4JtQNTI4X3KcbzacDJsMw,49.251342,-123.101333,3.5,169,1,0,1,0,0,...,False,False,False,False,False,False,False,1,0,0
3,jFYIsSb7r1QeESVUnXPHBw,39.947007,-82.997471,4.5,11,1,0,0,0,0,...,False,False,False,False,False,False,False,0,0,0
4,rYs_1pNB_RMtn5WQh55QDA,39.998449,-105.281006,3.5,5,1,1,0,0,0,...,False,False,False,False,False,False,False,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63910,Q78fYV6B6P6GmX07YVgi4g,30.304761,-97.937394,3.0,310,1,0,0,0,0,...,False,False,False,False,False,False,False,1,0,0
63911,uXdQkuEtvLAzfc3MsO-sTQ,33.810996,-84.367084,3.0,185,1,0,1,0,0,...,False,False,False,False,False,False,False,1,0,0
63912,bQX-kwVTyZgcdZGEPzce6Q,28.499836,-81.047478,4.5,18,1,0,0,0,0,...,False,False,False,False,False,False,False,0,0,0
63913,GB75wPibj3IjNauaoCxyGA,45.529647,-122.685153,4.0,10,1,0,0,0,0,...,False,False,False,False,False,False,False,0,0,0


#### Replacing the NoiseLevel column values from 'quiet','average', 'loud' and 'very_loud' to 0, 1, 2, and 3.</br>NoiseLevelcolumn is in the format like:</br>"u'average'"</br>So we use the eval function to correct this format

In [23]:
uniqueattributes =set()
for i in range(len(clean_business)):
    if (str(clean_business['NoiseLevel'][i]) != 'nan') and (str(clean_business['NoiseLevel'][i]) != 'None'):
        string = clean_business['NoiseLevel'][i]
        w = eval(string)
        uniqueattributes.add(w)
        
colvalue = []
for i in range(len(clean_business['NoiseLevel'])):
    string = clean_business['NoiseLevel'][i]
    if (str(clean_business['NoiseLevel'][i]) != 'nan') and (str(clean_business['NoiseLevel'][i]) != 'None'):
        w = eval(string)
        colvalue.append(w)
    else:
        colvalue.append(np.nan)
        
clean_business['NoiseLevel']=colvalue

clean_business['NoiseLevel'] = clean_business['NoiseLevel'].replace(['average','None',np.nan],1).replace('quiet',0).replace('loud',2).replace('very_loud',3)

#### We finally saved the clean_business dataset to use for further use.

In [24]:
# clean_business.to_pickle("./clean_business.pkl")

In [25]:
# having a quick look at the business dataset.
clean_business.head()

,business_id,latitude,longitude,stars,review_count,is_open,Food,Restaurants,Bars,American (Traditional),...,WheelchairAccessible,classy,upscale,trendy,divey,touristy,romantic,casual,intimate,hipster
0,6iYb2HFDywm3zjuRg0shjw,40.017544,-105.283348,4.0,86,1,0,0,0,0,...,True,False,False,False,False,False,False,False,False,False
1,tCbdrRPZA0oiIYSmHG3J0w,45.588906,-122.593331,4.0,126,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,D4JtQNTI4X3KcbzacDJsMw,49.251342,-123.101333,3.5,169,1,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
3,jFYIsSb7r1QeESVUnXPHBw,39.947007,-82.997471,4.5,11,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,rYs_1pNB_RMtn5WQh55QDA,39.998449,-105.281006,3.5,5,1,1,0,0,0,...,True,False,False,False,False,False,False,False,False,False
